# <b style="color: white; background-color: #00bbff; padding: 5px 10px; border-radius: 5px;">LIBRARY</b>

In [1]:
import re
import boto3
from botocore.client import Config
import os
import logging
import json
from dotenv import load_dotenv
from datetime import datetime

# -------------------------
# Configuração do Logging
# -------------------------
def setup_logger():
    # Cria o nome do arquivo de log com timestamp
    log_directory = "/opt/notebook/logs/"
    os.makedirs(log_directory, exist_ok=True)  # Garante que o diretório existe
    log_filename = f"etl_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    log_file = os.path.join(log_directory, log_filename)
    
    # Evita múltiplos handlers
    logger = logging.getLogger("minio_silver")
    if logger.handlers:  # Remove handlers existentes
        logger.handlers = []
    
    logger.setLevel(logging.INFO)
    
    # Formato do log
    formatter = logging.Formatter(fmt='{"level": "%(levelname)s", "message": "%(message)s"}')
    
    # Handler para console
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    
    # Handler para arquivo
    file_handler = logging.FileHandler(log_file)
    file_handler.setFormatter(formatter)
    
    # Adiciona ambos handlers
    logger.addHandler(console_handler)
    logger.addHandler(file_handler)
    
    return logger

logger = setup_logger()

In [2]:
# -------------------------
# Configurações
# -------------------------
MINIO_BUCKET = "ingestion"
DATA_FOLDER = "/opt/notebook/data"

# Carregar variáveis do arquivo .env
load_dotenv()

s3_endpoint = os.getenv("S3_ENDPOINT")
s3_access_key = os.getenv("S3_ACCESS_KEY")
s3_secret_key = os.getenv("S3_SECRET_KEY")

# Cria cliente do MinIO com boto3
s3 = boto3.client(
    "s3",
    endpoint_url=s3_endpoint,
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
    config=Config(signature_version="s3v4"),
    region_name="us-east-1"
)

def get_folder_prefix(filename):
    """
    Extrai o prefixo da pasta baseado no nome do arquivo:
    - Para 'vendas_part1.csv' → 'vendas/'
    - Para 'clientes.csv' → 'clientes/'
    - Padrão genérico para qualquer tabela
    """
    # Remove a extensão .csv
    base_name = filename.replace('.csv', '')
    
    # Padrão para arquivos particionados (ex: vendas_part1)
    match = re.match(r'^(.+)_part\d+$', base_name)
    if match:
        return f"{match.group(1)}/"
    
    # Se não for particionado, usa o nome base
    return f"{base_name}/"

# Verifica se o bucket existe
buckets = [bucket['Name'] for bucket in s3.list_buckets()['Buckets']]
if MINIO_BUCKET not in buckets:
    s3.create_bucket(Bucket=MINIO_BUCKET)
    logger.info(f"Bucket '{MINIO_BUCKET}' criado.")
else:
    logger.info(f"Bucket '{MINIO_BUCKET}' já existe.")

# Lista arquivos .csv
csv_files = [f for f in os.listdir(DATA_FOLDER) if f.endswith(".csv")]

# Se não houver arquivos, encerra
if not csv_files:
    logger.info("Nenhum arquivo .csv encontrado. Processo encerrado.")
    exit(0)
    
# Envia os arquivos e remove após envio
for filename in csv_files:
    filepath = os.path.join(DATA_FOLDER, filename)
    logger.info(f"Iniciando upload do arquivo: {filename}")

    try:
        # Determina o prefixo da pasta dinamicamente
        folder_prefix = get_folder_prefix(filename)
        s3_key = folder_prefix + filename
        
        with open(filepath, "rb") as f:
            s3.upload_fileobj(f, MINIO_BUCKET, s3_key)
        logger.info(f"Arquivo enviado com sucesso para {s3_key}")

        os.remove(filepath)
        logger.info(f"Arquivo removido após upload: {filename}")
    except Exception as e:
        logger.error(f"Erro ao processar {filename}: {str(e)}")

logger.info("Todos os arquivos CSV foram processados com sucesso.")

{"level": "INFO", "message": "Bucket 'ingestion' já existe."}
{"level": "INFO", "message": "Iniciando upload do arquivo: clientes.csv"}
{"level": "INFO", "message": "Arquivo enviado com sucesso para clientes/clientes.csv"}
{"level": "INFO", "message": "Arquivo removido após upload: clientes.csv"}
{"level": "INFO", "message": "Iniciando upload do arquivo: vendas.csv"}
{"level": "INFO", "message": "Arquivo enviado com sucesso para vendas/vendas.csv"}
{"level": "INFO", "message": "Arquivo removido após upload: vendas.csv"}
{"level": "INFO", "message": "Todos os arquivos CSV foram processados com sucesso."}
